Based on:

https://www.dominodatalab.com/blog/named-entity-recognition-ner-challenges-and-model

https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

# Install packages

In [1]:
!pip install plot_keras_history
!pip install keras
!pip install tensorflow
!pip install pyconll
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install conllu
!pip install flair

You should consider upgrading via the '/home/digitalhub/.pyenv/versions/3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/digitalhub/.pyenv/versions/3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
INFO: pip is looking at multiple versions of google-auth-oauthlib to determine which version is compatible with other requirements. This could take a while.
^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/home/digitalhub/.pyenv/versions/3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/digitalhub/.pyenv/versions/3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-bs906pag
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-bs906pag
  fatal: unable to access 'https://www.github.com/keras-team/keras-contri

# Import packages

In [2]:
import pickle
import operator
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyconll
import tensorflow as tf

from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from keras_contrib.utils import save_load_utils

from keras import layers
from keras import optimizers

from keras.models import Model
from keras.layers import Input

from keras_contrib.layers import CRF
from keras_contrib import losses
from keras_contrib import metrics

from typing import List
from flair.data import Corpus
from flair.datasets import ColumnCorpus, CONLL_03
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.embeddings import WordEmbeddings, TokenEmbeddings, FlairEmbeddings, PooledFlairEmbeddings, StackedEmbeddings

2022-11-30 10:56:29.732300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-30 10:56:29.732316: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# BiLSTM-CRF

## On domain-specific (re3d) data

In [4]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='val.conll')

2022-11-30 10:57:19,078 Reading data from entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits
2022-11-30 10:57:19,079 Train: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/train.conll
2022-11-30 10:57:19,079 Dev: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/val.conll
2022-11-30 10:57:19,079 Test: entity-recognition-datasets-master/data/re3d/CONLL-format/data/splits/test.conll


In [5]:
print(corpus.train[0].to_tagged_string('ner'))
print(len(corpus.train))

Sentence: "Operation Steel Curtain ( Arabic : ا ل ح ج ا ب ا ل ف و ل ا ذ ي Al Hejab Elfulathi ) was a military operation executed by coalition forces in early November 2005 to reduce the flow of foreign insurgents crossing the border and joining the Iraqi insurgency ." → ["coalition forces"/Organisation, "early November 2005"/Temporal, "foreign insurgents"/Organisation, "Iraqi"/Nationality]
771


In [6]:
# Define tags to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

2022-11-30 10:57:24,999 Computing label dictionary. Progress:


771it [00:00, 65857.65it/s]

2022-11-30 10:57:25,018 Dictionary created for label 'ner' with 11 values: Organisation (seen 1113 times), Location (seen 647 times), Person (seen 415 times), Temporal (seen 179 times), Quantity (seen 151 times), Weapon (seen 81 times), DocumentReference (seen 50 times), MilitaryPlatform (seen 48 times), Nationality (seen 34 times), Money (seen 20 times)


In [7]:
# Embeddings

from typing import List
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

In [8]:
embedding_types = [
    # init standard GloVe embedding
    WordEmbeddings('glove'),

    # init Flair forward and backwards embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)


In [9]:
from flair.models import SequenceTagger

# initialize a BiLSTM-CRF sequence tagger
tagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

from flair.trainers import ModelTrainer

# initialize trainer
trainer = ModelTrainer(tagger, corpus)

# start training
trainer.train('resources/taggers/re3d-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2022-11-29 14:33:10,012 SequenceTagger predicts: Dictionary with 41 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-Person, I-Person, S-Temporal, B-Temporal, E-Temporal, I-Temporal, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-Weapon, B-Weapon, E-Weapon, I-Weapon, S-DocumentReference, B-DocumentReference, E-DocumentReference, I-DocumentReference, S-MilitaryPlatform, B-MilitaryPlatform, E-MilitaryPlatform, I-MilitaryPlatform, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Money, B-Money, E-Money, I-Money
SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=

/home/digitalhub/.pyenv/versions/3.8.12/lib/python3.8/site-packages/flair/trainers/trainer.py:64: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2022-11-29 14:33:17,470 epoch 1 - iter 2/25 - loss 3.92332979 - samples/sec: 8.72 - lr: 0.100000
2022-11-29 14:33:24,642 epoch 1 - iter 4/25 - loss 3.12740240 - samples/sec: 8.92 - lr: 0.100000
2022-11-29 14:33:30,958 epoch 1 - iter 6/25 - loss 2.77152550 - samples/sec: 10.13 - lr: 0.100000
2022-11-29 14:33:38,714 epoch 1 - iter 8/25 - loss 2.51315506 - samples/sec: 8.25 - lr: 0.100000
2022-11-29 14:33:45,796 epoch 1 - iter 10/25 - loss 2.35552140 - samples/sec: 9.04 - lr: 0.100000
2022-11-29 14:33:53,326 epoch 1 - iter 12/25 - loss 2.23115748 - samples/sec: 8.50 - lr: 0.100000
2022-11-29 14:34:06,877 epoch 1 - iter 14/25 - loss 2.19633433 - samples/sec: 4.72 - lr: 0.100000
2022-11-29 14:34:15,095 epoch 1 - iter 16/25 - loss 2.11719954 - samples/sec: 7.79 - lr: 0.100000
2022-11-29 14:34:22,821 epoch 1 - iter 18/25 - loss 2.07095603 - samples/sec: 8.29 - lr: 0.100000
2022-11-29 14:34:31,439 epoch 1 - iter 20/25 - loss 2.05533246 - samples/sec: 7.43 - lr: 0.100000
2022-11-29 14:34:42,635

100%|██████████| 4/4 [00:11<00:00,  2.75s/it]

2022-11-29 14:35:02,376 Evaluating as a multi-label problem: False
2022-11-29 14:35:02,382 DEV : loss 1.4280058145523071 - f1-score (micro avg)  0.0056
2022-11-29 14:35:02,390 BAD EPOCHS (no improvement): 0
2022-11-29 14:35:02,391 saving best model


...

2022-11-29 14:58:56,113 Evaluating as a multi-label problem: False
2022-11-29 14:58:56,119 DEV : loss 0.27884408831596375 - f1-score (micro avg)  0.7094


2022-11-29 14:58:56,128 Epoch    80: reducing learning rate of group 0 to 9.7656e-05.
2022-11-29 14:58:56,128 BAD EPOCHS (no improvement): 4
2022-11-29 14:58:56,129 ----------------------------------------------------------------------------------------------------
2022-11-29 14:58:56,129 ----------------------------------------------------------------------------------------------------
2022-11-29 14:58:56,130 learning rate too small - quitting training!
2022-11-29 14:58:56,130 ----------------------------------------------------------------------------------------------------
2022-11-29 14:58:56,716 ----------------------------------------------------------------------------------------------------
2022-11-29 14:58:56,717 loading file resources/taggers/re3d-ner/best-model.pt
2022-11-29 14:58:57,277 SequenceTagger predicts: Dictionary with 43 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-P

100%|██████████| 3/3 [00:11<00:00,  3.71s/it]

2022-11-29 14:59:08,627 Evaluating as a multi-label problem: False
2022-11-29 14:59:08,633 0.6751	0.7492	0.7103	0.561
2022-11-29 14:59:08,634 
Results:
- F-score (micro) 0.7103
- F-score (macro) 0.4844
- Accuracy 0.561

By class:
                   precision    recall  f1-score   support

     Organisation     0.6605    0.7643    0.7086       140
         Location     0.6538    0.7500    0.6986        68
           Person     0.7879    0.8387    0.8125        62
         Temporal     0.9048    1.0000    0.9500        19
         Quantity     0.4118    0.4375    0.4242        16
DocumentReference     0.2500    0.2500    0.2500         4
           Weapon     0.0000    0.0000    0.0000         4
      Nationality     0.5000    0.5000    0.5000         2
 MilitaryPlatform     1.0000    0.3333    0.5000         3
            Money     0.0000    0.0000    0.0000         1

        micro avg     0.6751    0.7492    0.7103       319
        macro avg     0.5169    0.4874    0.4844       319
 

{'test_score': 0.7102526002971769,
 'dev_score_history': [0.0055710306406685246,
  0.028,
  0.2721311475409836,
  0.33203125,
  0.39528023598820056,
  0.5331179321486268,
  0.5800865800865801,
  0.5830618892508144,
  0.6053042121684867,
  0.6116207951070336,
  0.562111801242236,
  0.593619972260749,
  0.6295754026354319,
  0.6461988304093567,
  0.6463768115942029,
  0.6287339971550497,
  0.637329286798179,
  0.6142208774583964,
  0.6477272727272727,
  0.6557377049180328,
  0.6550218340611353,
  0.6483180428134556,
  0.6538461538461539,
  0.6496453900709219,
  0.6956521739130435,
  0.6787709497206704,
  0.7001434720229556,
  0.6930091185410335,
  0.692090395480226,
  0.6960486322188449,
  0.7073863636363638,
  0.7063829787234042,
  0.6870229007633588,
  0.693069306930693,
  0.6629680998613037,
  0.691114245416079,
  0.7080924855491331,
  0.6991404011461319,
  0.7020057306590258,
  0.6919831223628691,
  0.6881405563689604,
  0.6963788300835654,
  0.6899441340782123,
  0.6985507246376811,

## On general (docred) data

# Transformer model for domain-specific dataset

## On domain-specific (re3d) data

In [8]:
from flair.embeddings import TransformerWordEmbeddings

# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. run fine-tuning
trainer.fine_tune('resources/taggers/sota-ner-flert',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  )

2022-11-30 10:57:44,994 SequenceTagger predicts: Dictionary with 41 tags: O, S-Organisation, B-Organisation, E-Organisation, I-Organisation, S-Location, B-Location, E-Location, I-Location, S-Person, B-Person, E-Person, I-Person, S-Temporal, B-Temporal, E-Temporal, I-Temporal, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-Weapon, B-Weapon, E-Weapon, I-Weapon, S-DocumentReference, B-DocumentReference, E-DocumentReference, I-DocumentReference, S-MilitaryPlatform, B-MilitaryPlatform, E-MilitaryPlatform, I-MilitaryPlatform, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Money, B-Money, E-Money, I-Money
2022-11-30 10:57:44,999 ----------------------------------------------------------------------------------------------------
2022-11-30 10:57:45,000 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 1024, padding_idx=1)
        (position_embe

100%|██████████| 25/25 [00:26<00:00,  1.06s/it]

2022-11-30 11:14:46,124 Evaluating as a multi-label problem: False
2022-11-30 11:14:46,130 DEV : loss 1.2281770706176758 - f1-score (micro avg)  0.0106
2022-11-30 11:14:46,132 BAD EPOCHS (no improvement): 4
2022-11-30 11:14:46,133 ----------------------------------------------------------------------------------------------------


2022-11-30 11:16:25,474 epoch 2 - iter 19/193 - loss 1.70634697 - samples/sec: 0.77 - lr: 0.000005
2022-11-30 11:18:08,092 epoch 2 - iter 38/193 - loss 1.60078184 - samples/sec: 0.74 - lr: 0.000005
2022-11-30 11:19:53,055 epoch 2 - iter 57/193 - loss 1.52618754 - samples/sec: 0.72 - lr: 0.000005
2022-11-30 11:21:36,149 epoch 2 - iter 76/193 - loss 1.47910050 - samples/sec: 0.74 - lr: 0.000005
2022-11-30 11:23:18,441 epoch 2 - iter 95/193 - loss 1.48425810 - samples/sec: 0.74 - lr: 0.000005
2022-11-30 11:24:58,056 epoch 2 - iter 114/193 - loss 1.48248315 - samples/sec: 0.76 - lr: 0.000005
2022-11-30 11:26:40,738 epoch 2 - iter 133/193 - loss 1.47193090 - samples/sec: 0.74 - lr: 0.000005
2022-11-30 11:28:24,931 epoch 2 - iter 152/193 - loss 1.45062939 - samples/sec: 0.73 - lr: 0.000005
2022-11-30 11:30:03,297 epoch 2 - iter 171/193 - loss 1.42916158 - samples/sec: 0.77 - lr: 0.000005
2022-11-30 11:31:45,988 epoch 2 - iter 190/193 - loss 1.41856586 - samples/sec: 0.74 - lr: 0.000004
2022-

100%|██████████| 25/25 [00:27<00:00,  1.09s/it]

...

2022-11-30 13:55:45,704 Evaluating as a multi-label problem: False
2022-11-30 13:55:45,711 DEV : loss 0.4879798889160156 - f1-score (micro avg)  0.6458
2022-11-30 13:55:45,713 BAD EPOCHS (no improvement): 4


2022-11-30 13:55:47,663 ----------------------------------------------------------------------------------------------------
2022-11-30 13:55:47,665 Testing using last state of model ...


100%|██████████| 24/24 [00:25<00:00,  1.07s/it]

2022-11-30 13:56:13,300 Evaluating as a multi-label problem: False
2022-11-30 13:56:13,306 0.6446	0.7618	0.6983	0.5473
2022-11-30 13:56:13,307 
Results:
- F-score (micro) 0.6983
- F-score (macro) 0.3655
- Accuracy 0.5473

By class:
                   precision    recall  f1-score   support

     Organisation     0.6606    0.7786    0.7148       140
         Location     0.5934    0.7941    0.6792        68
           Person     0.7568    0.9032    0.8235        62
         Temporal     0.7826    0.9474    0.8571        19
         Quantity     0.2778    0.3125    0.2941        16
DocumentReference     0.3333    0.2500    0.2857         4
           Weapon     0.0000    0.0000    0.0000         4
            Money     0.0000    0.0000    0.0000         1
 MilitaryPlatform     0.0000    0.0000    0.0000         3
      Nationality     0.0000    0.0000    0.0000         2

        micro avg     0.6446    0.7618    0.6983       319
        macro avg     0.3404    0.3986    0.3655       319

{'test_score': 0.6982758620689655,
 'dev_score_history': [0.010554089709762533,
  0.33587786259541985,
  0.5598845598845599,
  0.5923984272608126,
  0.6229508196721312,
  0.6420765027322404,
  0.6610169491525424,
  0.6418732782369146,
  0.6481994459833795,
  0.6457765667574932],
 'train_loss_history': [2.9750299324347216,
  1.4151145429336154,
  1.102379407128485,
  0.8985091482578567,
  0.7659444744710038,
  0.7034005997481789,
  0.6549618372841,
  0.6058066446442902,
  0.57429869816408,
  0.5604896890343632],
 'dev_loss_history': [1.2281770706176758,
  0.859157383441925,
  0.6455650925636292,
  0.5550165176391602,
  0.5053684711456299,
  0.4809141457080841,
  0.45765113830566406,
  0.4889335632324219,
  0.46722859144210815,
  0.4879798889160156]}